In [50]:
import symforce

symforce.set_symbolic_api("sympy")
symforce.set_log_level("warning")

from symforce.notebook_util import display
import symforce.symbolic as sf

def readTrajectory(path):

    trajectory = []

    with open(path) as f:
        for line in f:
            components = line.split(" ")
            time, tx, ty, tz, qx, qy, qz, qw = map(lambda x: float(x), components)
            rot = sf.Rot3(sf.Quaternion(sf.V3(qx, qy, qz), qw))
            trans = sf.V3(tx,ty,tz)
            p = sf.Pose3(R = rot, t= trans)
            trajectory.append(p)

    return trajectory
            
trajectory = readTrajectory('./trajectory.txt')
estimated = readTrajectory('./estimated.txt')
groundtruth = readTrajectory('./groundtruth.txt')

print("Loaded trajectories.")

Loaded trajectories.


In [53]:
# compute RMSE
from math import sqrt
rmse = 0

for i in range(len(estimated)):
    p1, p2 = estimated[i], groundtruth[i]
    error = sf.V6((p2.inverse() * p1).to_tangent()).norm()
    rmse += error ** 2

rmse /= len(trajectory)
rmse = sqrt(rmse)

print("RMSE: %s"%rmse)

RMSE: 2.1929347657113603


In [54]:
import open3d as o3d

def gen_line_set(trajectory, plotFrames=True, traj_color=[0, 0, 0]):
    """Returns line_set"""
    index = 0
    points = []
    lines = []
    colors = []
    for traj in trajectory:
        length = 0.1
        Ow = traj.t
        Xw = traj * (length*sf.V3(1, 0, 0))
        Yw = traj * (length*sf.V3(0, 1, 0))
        Zw = traj * (length*sf.V3(0, 0, 1))
        points.append(Ow.to_flat_list())
        points.append(Xw.to_flat_list())
        points.append(Yw.to_flat_list())
        points.append(Zw.to_flat_list())

        if plotFrames:
            lines.append([index, index + 1])
            lines.append([index, index + 2])
            lines.append([index, index + 3])

            colors.append([1, 0, 0])
            colors.append([0, 1, 0])
            colors.append([0, 0, 1])

        index = index + 4

    index = 0
    for traj in trajectory:
        if index == 0:
            pass
        else:
            lines.append([index, index - 4])
            colors.append(traj_color)
        index = index + 4

    line_set = o3d.geometry.LineSet(
        points=o3d.utility.Vector3dVector(points),
        lines=o3d.utility.Vector2iVector(lines),
    )
    line_set.colors = o3d.utility.Vector3dVector(colors)
    return line_set

line_set1 = gen_line_set(estimated, False, [0, 0, 1])  # blue
line_set2 = gen_line_set(groundtruth, False, [1, 0, 0])  # red
vis = o3d.visualization.Visualizer()
vis.create_window()
vis.add_geometry(line_set1)
vis.add_geometry(line_set2)
vis.add_geometry(o3d.geometry.TriangleMesh.create_coordinate_frame(size=.2))
while(True):
    if not vis.poll_events():
        break
    vis.update_renderer()
vis.destroy_window()


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
